In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
sb.set(style='white')
sb.set(style='whitegrid', color_codes= True)
# Any results you write to the current directory are saved as output.


In [ ]:
path_train ='../input/train.csv'
df = pd.read_csv(path_train,header=0)
y = df.Survived
# features = ["Age", "Pclass", "Fare", "Cabin", "Sex"]
sb.set_context(context='talk')

#for feature = Pclass
sb.regplot(x='Pclass', y='Survived', data = df)

#for feature = embarked
temp_embarked_col = df['Embarked']
temp_embarked_col = temp_embarked_col.dropna()
sb.pointplot(temp_embarked_col, y='Survived', data=df)
#NOTE: THIS SHOWS THAT PASSENGERS ON CLASS c HAVE HIGHER CHANCES OF SURVIVAL



In [ ]:
#for feature = sex
#women = df[df['Sex']=='female']
#men = df[df['Sex']=='male']
#sb.distplot(women)

sb.barplot(x='Sex', y='Survived', data=df)

#for feature = Age
temp_age = df['Age'].dropna()
g = sb.FacetGrid(df, col="Survived")
g = g.map(plt.hist, "Age")

#for sibsp
sb.barplot(x='SibSp', y='Survived', data=df)

#for parch
sb.barplot(x='Parch', y='Survived', data=df)

#for fare 
temp_fare = df['Fare'].dropna()
sb.distplot(temp_fare, bins=100)

In [ ]:
#print(df)
df.info()

In [ ]:
#imputing missing values
#to interpolate all the missing values in the age column
df['Age'] = df['Age'].interpolate()
# Embarked replace NaN with the mode value
ModeEmbarked = df['Embarked'].mode()
df.Embarked = df.Embarked.fillna(value=ModeEmbarked)
# Fare have 1 NaN missing value on the Submission dataset
MedianFare = df.Fare.median()
df.Fare = df.Fare.fillna(value=MedianFare)

In [ ]:
df.info()

In [ ]:
#encoding the categorical features
df['Cabin'] = df.apply(lambda obs: 'No' if pd.isnull(obs['Cabin']) else 'Yes',axis=1)
df = pd.get_dummies(df, columns=['Sex','Cabin', 'Embarked'])

In [ ]:
#for encoding cells
df.info()

In [ ]:
#scaling the continuous variables numerically
scale = StandardScaler().fit(df[['Age', 'Fare']])
df[['Age','Fare']] = scale.transform(df[['Age','Fare']])

In [ ]:
#dropping the features that are too categorical and do not contribute to our model
df = df.drop(['Survived','Name','Ticket','PassengerId'],axis=1)
df.info()

In [ ]:
#cleaning and transforming the test data

#creating test and train sets
path_test ='../input/test.csv'
test = pd.read_csv(path_test,header=0)
test_passengers = test['PassengerId']
predictor_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Cabin', 'Embarked']
test = test[predictor_cols]

#imputing missing values
#to interpolate all the missing values in the age column
test['Age'] = test['Age'].interpolate()
# Embarked replace NaN with the mode value
ModeEmbarked = test['Embarked'].mode()
test.Embarked = test.Embarked.fillna(value=ModeEmbarked)
# Fare have 1 NaN missing value on the Submission dataset
MedianFare = test.Fare.median()
test.Fare = test.Fare.fillna(value=MedianFare)

#encoding the categorical features
test['Cabin'] = test.apply(lambda obs: 'No' if pd.isnull(obs['Cabin']) else 'Yes',axis=1)
test = pd.get_dummies(test, columns=['Sex','Cabin', 'Embarked'])

#scaling the continuous variables numerically
scale = StandardScaler().fit(test[['Age', 'Fare']])
test[['Age','Fare']] = scale.transform(test[['Age','Fare']])

test.info()

In [ ]:
# here come the ml algorithms
#converting the dataframe into numpy
train = df.values
train_output = y.values


MlRes= {}
def MlResult(model,score):
    MlRes[model] = score
    print(MlRes)

#using logistic regression
logreg = LogisticRegression()
#add the regression data
logreg.fit(train, train_output)
#using the test data to predict on the model
predicted_result = logreg.predict(test)
print(predicted_result)
#show the score/metrics
#accuracy = logreg.score(train_output, test_output)
#MlResult('Logistic Regression', accuracy)




In [ ]:
#submission code
submission = pd.DataFrame({'PassengerId':test_passengers,'Survived':predicted_result})
submission.to_csv('submission.csv', index=False)
# print(test_passengers)

    